In [1]:
import sys,os,re
import glob
import pandas as pd
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,udf,lower

In [2]:
spark = SparkSession \
        .builder \
        .master("local[*]") \
        .config("spark.executor.memory", "70g") \
        .config("spark.driver.memory", "50g") \
        .config("spark.memory.offHeap.enabled",True) \
        .config("spark.memory.offHeap.size","16g") \
        .appName("Airline") \
        .getOrCreate()

22/11/03 09:25:44 WARN Utils: Your hostname, LinuxGUI resolves to a loopback address: 127.0.1.1; using 192.168.0.103 instead (on interface eno1)
22/11/03 09:25:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/03 09:25:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# Read all airline delay info under ./data
US_ALs = spark.read.option("header",True).csv('./data/airlines/')
US_APs = spark.read.option("header",True).csv('./data/us-airports_clean.csv')
Stations = spark.read.option("header",False).csv('./data/ghcnd-stations_clean.csv')
# airportDF = spark.read.option("header",True).csv('./data/airport-codes_clean.csv')

US_ALs.createOrReplaceTempView("Airlines")
US_APs.createOrReplaceTempView("Airports")
Stations.createOrReplaceTempView("Stations")

# Create UNIQUE airport identifier in dataset
AP_CODEs = spark.sql(" \
    SELECT DISTINCT ORIGIN FROM AIRLINES \
    UNION \
    SELECT DISTINCT DEST FROM AIRLINES" \
)

# Create a table for it
AP_CODEs.createOrReplaceTempView("AirportCode")

In [ ]:
APIDNAMEs = spark.read.option("header",False).csv('./data/apIdName.csv')
APIDNAMEs.createOrReplaceTempView("APIDNAME")
APIDNAMELOCAL = spark.sql(" \
        SELECT * FROM APIDNAME AS n \
        LEFT JOIN Airports AS p ON n._c0 = p.local_code \
    ")

In [ ]:
# (ORIGIN, STATE, CITY, AP LOCAL_CODE, AP IATA_CODE, AP IDENT_CODE, AP NAME)
FULL_DF = spark.sql(" \
        SELECT c.ORIGIN, \
            p.local_region AS STATE, \
            p.municipality AS CITY, \
            p.local_code AS LOCAL, \
            p.iata_code AS IATA, \
            p.ident AS IDENT, \
            p.name AS NAME \
        FROM AirportCode AS c \
        LEFT JOIN Airports AS p ON \
        (c.ORIGIN = p.local_code OR c.ORIGIN = p.iata_code)" \
    )
    
FULL_DF.write.option("header",True).csv('./output/full')
FULL_DF.createOrReplaceTempView("FULL_AP")

In [ ]:
sqlStr = "SELECT * FROM FULL_AP AS f \
    LEFT JOIN Stations AS s "

# MATCH_STATIONS_DF = spark.sql()

ap_stations = Stations.filter( \
    lower(col('_c2')).like('% ap%') | \
    lower(col('_c2')).like('% airport%')) \

ap_stations.createOrReplaceTempView("APTABLE")
ap_stations.write.csv('./output/ap_station')


sqlStr = "SELECT * FROM APTABLE AS t \
        LEFT JOIN FULL_AP AS p ON \
        t._c2 LIKE CONCAT('%', p.column ,'%') "

In [ ]:
# Version2: Output AirportID,AirportName into ./output/airports.csv
spark.sql(" \
    SELECT i.ORIGIN, p.name \
    FROM AIRPORTID i \
    LEFT JOIN \
    AIRPORTS p ON i.ORIGIN = ( \
    CASE WHEN (p.iata_code IS NOT NULL) THEN (p.iata_code) \
    ELSE (p.local_code) END) \
    WHERE p.iso_country = 'US'" \
    ) \
    .sort("ORIGIN") \
    .write.options(heade=True, delimiter=',') \
    .csv("./output/airports")

In [ ]:
# load stations info
apIdNameDF = spark.read.option("header",False).csv('./data/apIdName.csv')
apIdNames = apIdNameDF.collect()

name_seg = set()
for s in apIdNames:
    for i in str(s[1]).split():
        if i.isalpha():
            name_seg.add(i)

tl = list(name_seg)
tl.sort()
print(tl)

In [ ]:
ghcndDF = spark.read.option("header",False).csv('./data/ghcnd-stations_clean.csv')
potentials = ghcndDF.filter( \
    lower(col('_c2')).like('% ap%') | \
    lower(col('_c2')).like('% airport%')).write.csv('test.csv')

In [3]:
full_airport_station =  spark.read.option("header",True).csv('./data/airports_stations.csv')
ghcnd_stations = spark.read.option("header",False).csv('./data/ghcnd_by_year/')

full_airport_station.createOrReplaceTempView('full_airport_station')
ghcnd_stations.createOrReplaceTempView('ghcnd_stations')

print(ghcnd_stations.count())
print(full_airport_station.head())
print(ghcnd_stations.head())

374901618
Row(ORIGIN='BGM', STATE='NY', CITY='Binghamton', LOCAL='BGM', IATA='BGM', IDENT='KBGM', NAME='Greater Binghamton/Edwin A Link field', STATION='USW00014738')
Row(_c0='AE000041196', _c1='20130101', _c2='TMAX', _c3='250', _c4=None, _c5=None, _c6='S', _c7=None)


In [5]:
selected_airport_station = full_airport_station.select(col('ORIGIN'),col('STATION'))
ghcnd_stations_clean = ghcnd_stations.join( \
                            selected_airport_station, \
                            ghcnd_stations._c0 == selected_airport_station.STATION, \
                            "LeftOuter" \
                        )

In [8]:
# Keep not null rows and rename them based on GHCND documents
filter_null = ghcnd_stations_clean \
                .filter(ghcnd_stations_clean.ORIGIN.isNotNull()) \
                .select(col('_c0').alias("STATION_ID"), \
                        col('ORIGIN').alias("AIRPORT_CODE"), \
                        col('_c1').alias("DATE"), \
                        col('_c2').alias("ELEMENT"), \
                        col('_c3').alias("DATA_VALUE"), \
                        col('_c4').alias("M_FLAG"), \
                        col('_c5').alias("Q_FLAG"), \
                        col('_c6').alias("S_FLAG"), \
                        col('_c7').alias("OBS_TIME"))
filter_null.count()
filter_null.write.option("header",True).csv("./output/filter_all_v4")